In [1]:
import numpy as np
from sklearn.impute import SimpleImputer
import sys
sys.path.append("..")
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import sys
import missing_process.missing_method as missing_method
from missing_process.block_rules import *
from utils import load_train_test,make_plot,RMSE,mask_check
import json
import matplotlib.pyplot as plt
import seaborn as sns



In [2]:
real_datalist = ["banknote","concrete_compression",
            "wine_quality_white","wine_quality_red",
            "california","climate_model_crashes",
            "connectionist_bench_sonar","qsar_biodegradation",
            "yeast","yacht_hydrodynamics"
            ]
#real_datalist = ["yacht_hydrodynamics"]

syn_datalist = ["syn1"]


missingtypelist = ["quantile","diffuse","logistic"]
#missingtypelist = ["diffuse"]

In [3]:
def mean_imp(missingtypelist,datalist_name,seed = 1,nfold = 5):


    if datalist_name == "real":
        datalist = real_datalist
    elif datalist_name == "syn":
        datalist = syn_datalist

    for missingtype in missingtypelist:
        print(missingtype)
        if missingtype == "logistic":
            missing_rule = load_json_file("missing_rate.json")
        elif missingtype == "diffuse":
            missing_rule = load_json_file("diffuse_ratio.json")
        elif missingtype == "quantile":
            missing_rule = load_json_file("quantile_full.json")



        for dataname in datalist:
            print(dataname)
            path = f"../impute/{missingtype}/{dataname}/zero"
            if not os.path.exists(path):
                # If the path does not exist, create it
                os.makedirs(path)

            for rule_name in missing_rule:

                    directory_path = f"../datasets/{dataname}"  
                    # Opening JSON file
                    f = open(f'{directory_path}/split_index_cv_seed-{seed}_nfold-{nfold}.json')
                    index_file = json.load(f)

                    for fold in index_file:

                        train_values,train_masks,test_values,test_masks = load_train_test(index_file[fold],missingtype,rule_name,directory_path,dataname)

                        train_values_na = np.where(train_masks == 0, np.nan, train_values)
                        test_values_na = np.where(test_masks == 0, np.nan, test_values)
                        
                        imp_mean = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)
                        imp_mean.fit(train_values_na)
                        test_imp = imp_mean.transform(test_values_na)
                        train_imp = imp_mean.transform(train_values_na)


                        np.save(f'{path}/{rule_name}_seed-{seed}_{fold}_train.npy', train_imp.astype("float32"))
                        np.save(f'{path}/{rule_name}_seed-{seed}_{fold}_test.npy', test_imp.astype("float32"))

                        if  (test_imp.shape != test_values_na.shape):
                            print("test shape")
                        if  (train_imp.shape != train_values_na.shape):
                            print("train shape")


            

In [4]:
real_datalist = [
            "banknote",
             "concrete_compression",
             "wine_quality_white",
            "wine_quality_red",
            "california",
            "climate_model_crashes",
            "connectionist_bench_sonar",
            "qsar_biodegradation", 
            "yeast", 
            "yacht_hydrodynamics"
            ]
missingtypelist = ["quantile",
                   "diffuse",
                   "logistic"
                   ]

mean_imp(missingtypelist,"real")

quantile
banknote
concrete_compression
wine_quality_white
wine_quality_red
california
climate_model_crashes
connectionist_bench_sonar
qsar_biodegradation
yeast
yacht_hydrodynamics
diffuse
banknote
concrete_compression
wine_quality_white
wine_quality_red
california
climate_model_crashes
connectionist_bench_sonar
qsar_biodegradation
yeast
yacht_hydrodynamics
logistic
banknote
concrete_compression
wine_quality_white
wine_quality_red
california
climate_model_crashes
connectionist_bench_sonar
qsar_biodegradation
yeast
yacht_hydrodynamics
